In [5]:
import json
from collections import defaultdict
import simplekml

In [6]:
# geometry source file supplied on the open data portal
fname = 'Voting_Subdivision.geojson'
with open(fname) as f:
    poll = json.load(f)

# polling station points supplied on the open data portal
stations = 'Voting_Polling_Stations.geojson'
with open(stations) as p:
    s_points = json.load(p)

In [7]:
# template to convert to KML - create a simplified geojson structure in case doing so is helpful for
# a future project
structure = {
    'type' : 'FeatureCollection',
    'name' : 'Voting_Subdivision_4Kml',
    'crs' : { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" }},
    'features' : []
    
}

In [8]:
# these are the polls with the highest amount of voter turnout - they are structured as (ward, poll)
targets = [(8,30),(8,15),(1,30),(2,10),(1,15),(7,30),(9,5),(2,25),(8,35),(3,15),(3,25),(2,5),(2,15),(2,60),(7,35),
          (7,40),(4,15),(7,25),(7,45)]

targets.sort(key=lambda x: x[0])

# for the sake of convenience create a dictionary to hold a list of the target polls in each ward
lu = defaultdict(list)
for t in targets:
    w,p = t
    lu[w].append(p)

In [9]:
# highlight target areas of the map and modify the geojson object

for p in poll['features']:
    # print a list of all the polls in each ward
    #print(f"{p['properties']['WARD']} {p['properties']['POLLING_STATION_NAME']} {p['properties']['POLL']}")
    pl = p['properties']['POLL']
    wd = p['properties']['WARD']
    
    # this will be the name of the polygon in google maps when you click it
    w_p_name = f"Ward: {wd} Poll: {pl}" 
    p['properties']['w_p_name'] = w_p_name
 
    # check for priority status
    is_tgt = lu.get(int(wd),[None])
    if int(pl) in is_tgt:
        structure['features'].append(p)
    else:
        pass # perhaps do something here later

# verify that we have the correct targets
for new_p in structure['features']:
    print(f"{new_p['properties']['WARD']} {new_p['properties']['POLLING_STATION_NAME']} {new_p['properties']['POLL']}")

1 MACKENZIE KING PUBLIC SCHOOL 15
1 SMITHSON PUBLIC SCHOOL 30
2 GRAND RIVER RECREATION COMPLEX 10
2 LACKNER WOODS PUBLIC SCHOOL 25
2 ST. JOHN PAUL II CATHOLIC SCHOOL 60
2 CHICOPEE HILLS PUBLIC SCHOOL 15
2 STANLEY PARK COMMUNITY CENTRE 5
3 K.P.L. - COUNTRY HILLS COMMUNITY LIBRARY 15
3 SPORTSWORLD ARENA 25
4 J.W. GERTH PUBLIC SCHOOL 15
7 K.P.L. - FOREST HEIGHTS COMMUNITY LIBRARY 35
7 DRIFTWOOD PARK PUBLIC SCHOOL 40
7 JOHN DARLING PUBLIC SCHOOL 45
7 FOREST HEIGHTS COMMUNITY CENTRE 30
7 ST. DOMINIC SAVIO CATHOLIC SCHOOL 25
8 WESTMOUNT PUBLIC SCHOOL 15
8 FOREST HILL PUBLIC SCHOOL 35
8 ST. PAUL CATHOLIC SCHOOL 30
9 ST. JOHN'S CATHOLIC SCHOOL 5


In [10]:
kml = simplekml.Kml()

In [12]:
# iterate over the structure and add the label and geometry to the kml instance

for feature in structure['features']:
    geo = feature['geometry']
    kml.newpolygon(name= feature['properties']['w_p_name'],
                   description =  feature['properties']['POLLING_STATION_NAME'],
                   outerboundaryis=geo['coordinates'][0]
                  )

for point in s_points['features']:
    kml_p = point['properties']['MAP_LABEL']
    gx,gy = point['geometry']['coordinates']
    kml.newpoint(name = kml_p,
                 coords = [(gx,gy)])
    

# save file for importing into google maps
kml.save('target_wards.kml')